In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!git clone https://github.com/valkryhx/localGPT

In [ ]:
%cd localGPT/
!pip install -r requirements.txt

In [ ]:
# cat /kaggle/working/localGPT/constants.py

In [ ]:
!git pull --all --force
!python ingest.py

# 上面这个max_se_length 其实是https://github.com/langchain-ai/langchain/blob/0689628489967785f3a11a9f29d8f6f90930f4f4/libs/langchain/langchain/embeddings/huggingface.py#L231C9-L231C65
Breadcrumbslangchain/libs/langchain/langchain/embeddings
/huggingface.py 231行sentence_transformers.SentenceTransformer 加载SentenceTransformer 默认的512  要动的话需要动源码

# ls DB/  ingest 之后的向量文件在这里  如果不想要就 rm -rf DB 那下次就要重新ingest

In [ ]:
#run_localGPT.py 中的query=input()无法在kaggle环境上正常执行 我们自己把代码拿出来跑
#!python run_localGPT.py

In [ ]:
# codes are just from local_GPT.py
import logging

import click
import torch
from auto_gptq import AutoGPTQForCausalLM
from huggingface_hub import hf_hub_download
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.llms import HuggingFacePipeline, LlamaCpp
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from loguru import logger
# from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.vectorstores import Chroma
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    GenerationConfig,
    LlamaForCausalLM,
    LlamaTokenizer,
    pipeline,
)

from constants import CHROMA_SETTINGS, EMBEDDING_MODEL_NAME, PERSIST_DIRECTORY, MODEL_ID, MODEL_BASENAME

In [ ]:
# 这里会下载和加载chatglm2-6b
!git pull --all --force
from my_chatglm_llm import ChatGLM

In [ ]:
h=[("123"),("3")]
sum([len(item) for item in h])

In [ ]:
!git pull --all --force

# collection_metadata={"hnsw:space": "cosine"}, # 重要add 20230831 将默认的L2 distance换成 cosine similarity

In [ ]:

device_type="cuda"
show_sources =True
EMBEDDING_MODEL_NAME = 'moka-ai/m3e-base'
logging.info(f"Running on: {device_type}")
logging.info(f"Display Source Documents set to: {show_sources}")
logger.error(f"EMBEDDING_MODEL_NAME={EMBEDDING_MODEL_NAME}")
embeddings = HuggingFaceInstructEmbeddings(model_name=EMBEDDING_MODEL_NAME, model_kwargs={"device": device_type})

# uncomment the following line if you used HuggingFaceEmbeddings in the ingest.py
# embeddings = HuggingFaceEmbeddings(model_name=EMBEDDING_MODEL_NAME)

# load the vectorstore
db = Chroma(
        persist_directory=PERSIST_DIRECTORY,
        embedding_function=embeddings,
        client_settings=CHROMA_SETTINGS,
        collection_metadata={"hnsw:space": "cosine"}, # 重要add 20230831 将默认的L2 distance换成 cosine similarity
    )
#retriever = db.as_retriever()

retriever = db.as_retriever(
    search_type="similarity_score_threshold", 
    search_kwargs={"k":3, "score_threshold":0.75}
    )

In [ ]:
template = """使用如下信息回答问题. 如果不知道答案,\
    就回答不知道，不要编造答案.

    {context}

    {history}
    问: {question}
    答:"""

prompt = PromptTemplate(input_variables=["history", "context", "question"], template=template)
memory = ConversationBufferMemory(input_key="question", memory_key="history")
logger.error(isinstance(memory,list)) # memory 不是 list
#print(memory)
#llm = load_model(device_type, model_id=MODEL_ID, model_basename=MODEL_BASENAME)
llm = ChatGLM()
qa = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        return_source_documents=True,
        chain_type_kwargs={"prompt": prompt, "memory": memory},
    )
    # Interactive questions and answers
cnt = 0

In [ ]:
# 注意  打印history的语句在chatglm_llm.py最下方 那里也可以调节history的取值 我现在取值history[-2:0]
while cnt == 0:
        #cnt += 1
        query = input("输入问题:\n")
        #query = "how long is the period of united states president?"
        #query = "SSE报销预览打印错误怎么办"
        if query == "exit":
            break
        # Get the answer from the chain
        res = qa(query)
        print("12345")
        answer, docs = res["result"], res["source_documents"]

        # Print the result
        print("\n\n> Question:")
        print(query)
        print("\n> Answer:")
        print(answer)

        if show_sources:  # this is a flag that you can set to disable showing answers.
            # # Print the relevant sources used for the answer
            print("----------------------------------SOURCE DOCUMENTS---------------------------")
            for document in docs:
                print("\n> " + document.metadata["source"] + ":")
                print(document.page_content)
            print("----------------------------------SOURCE DOCUMENTS---------------------------")

#  Chroma和FAISS默认的相似度计算metric是L2 distance
在Chroma.from_documents方法中加入参数
collection_metadata={"hnsw:space": "cosine"} 改成cosine 相似度 在[0,1]之间 越高说明两个向量越相似

Is you are using Chroma, you should set the distance metric when creating a collection: https://docs.trychroma.com/usage-guide#changing-the-distance-function

The default distance is l2. That is why for me it used to give scores like 3626.016357421875 when using the function similarity_search_with_relevance_scores(). On changing it to cosine, the scores are now between (0, 1] with scores closer to 1 depicting higher similarity.

Chroma.from_documents(documents=documents, embedding=cohere, collection_metadata={"hnsw:space": "cosine"})

参考 https://stackoverflow.com/questions/76678783/langchains-chroma-vectordb-similarity-search-with-score-and-vectordb-simil
参考 langchain.vectorstors.Chroma 源码  https://api.python.langchain.com/en/latest/_modules/langchain/vectorstores/chroma.html
的class Chroma类的__init__方法

# https://github.com/langchain-ai/langchain/issues/6481
# https://docs.trychroma.com/usage-guide#changing-the-distance-function
#注意这个 https://stackoverflow.com/questions/76678783/langchains-chroma-vectordb-similarity-search-with-score-and-vectordb-simil
# https://github.com/langchain-ai/langchain/issues/5458  
这个写了怎么在向量计算结果中加入相似度阈值和个数阈值进行过滤 结果不超过k个 并且score_threshold要大于这个0.5才参与候选
retriever=db.as_retriever(search_type="similarity_score_threshold", 
                          search_kwargs={"k":3, "score_threshold":0.5})    
https://github.com/langchain-ai/langchain/blob/e60e1cdf23ad73b2e0a40034c0ddfc3c8b0c9c4d/libs/langchain/langchain/vectorstores/base.py#L460

# langchain使用样例
https://python.langchain.com/docs/use_cases/question_answering/how_to/local_retrieval_qa

# python 合并字典 优雅
https://segmentfault.com/a/1190000010567015

In [ ]:
# 用新版本的https://github.com/valkryhx/localGPT   
# branch localGPT_0831_langchain_v02

In [ ]:
rm -rf /kaggle/working/localGPT

# <font color=red>注意 要让Chroma使用cosine distance（注意不是cosine similarity）而非默认的L2 distance</font>  
https://github.com/valkryhx/localGPT/blob/localGPT_0831_langchain_v02/ingest.py#L150 加一行   collection_metadata={"hnsw:space": "cosine"},
在下面的# load the vectorstore
db = MyChroma(
        persist_directory=PERSIST_DIRECTORY,
        embedding_function=embeddings,
        client_settings=CHROMA_SETTINGS,
        collection_metadata={"hnsw:space": "ip"}, # 重要add 20230831 将默认的L2 distance换成 cosine similarity
          
)
也加一行

# 魔改了langchain/vectorstores/chroma.py 增加了distance value输出  越相似的distance越小
# #retriever = db.as_retriever() 改了 增加 search_type="similarity_score_threshold", search_kwargs={"k":3, "score_threshold":score_threshold}
score_threshold = 0.6
retriever = db.as_retriever(
    search_type="similarity_score_threshold", 
    search_kwargs={"k":3, "score_threshold":score_threshold}
    )
 # 注意 这个threshold如果取0.7 那么实际对应的是distance value小于0.3的text 也就是说distance越小 则1-distance 越大 越容易超过threshold  这符合逻辑和变量名定义  后面print时 我会用1-distance的值来显示

In [ ]:
%cd /kaggle/working
!git clone -b localGPT_0831_langchain_v02 https://github.com/valkryhx/localGPT  

In [ ]:
%cd localGPT
!git status

In [ ]:
!git pull --all --force
!pip install -r requirements.txt

In [ ]:
#!pip list

In [ ]:
!git pull --all --force
!python ingest.py

In [ ]:
# 重新生成   
#ls DB
#!rm -rf DB

In [ ]:
# codes are just from local_GPT.py
import logging

import click
import torch
#from auto_gptq import AutoGPTQForCausalLM
from huggingface_hub import hf_hub_download
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.llms import HuggingFacePipeline, LlamaCpp
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from loguru import logger
# from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.vectorstores import Chroma
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    GenerationConfig,
    LlamaForCausalLM,
    LlamaTokenizer,
    pipeline,
)

from constants import CHROMA_SETTINGS, EMBEDDING_MODEL_NAME, PERSIST_DIRECTORY, MODEL_ID, MODEL_BASENAME

In [ ]:
# 这里会下载和加载chatglm2-6b
!git pull --all --force
from my_chatglm_llm import ChatGLM

根据这个问题

https://github.com/langchain-ai/langchain/issues/4710

https://github.com/langchain-ai/langchain/issues/5416

改的_results_to_docs_and_scores方法中
metadata={**result[1],**{"distances":result[2]} } or {})

In [ ]:
"""Wrapper around ChromaDB embeddings platform."""
from __future__ import annotations

import logging
import uuid
from typing import (
    TYPE_CHECKING,
    Any,
    Callable,
    Dict,
    Iterable,
    List,
    Optional,
    Tuple,
    Type,
)

import numpy as np

from langchain.docstore.document import Document
from langchain.embeddings.base import Embeddings
from langchain.utils import xor_args
from langchain.vectorstores.base import VectorStore
from langchain.vectorstores.utils import maximal_marginal_relevance

if TYPE_CHECKING:
    import chromadb
    import chromadb.config
    from chromadb.api.types import ID, OneOrMany, Where, WhereDocument

logger = logging.getLogger()
DEFAULT_K = 4  # Number of Documents to return.


def _results_to_docs(results: Any) -> List[Document]:
    return [doc for doc, _ in _results_to_docs_and_scores(results)]


def _results_to_docs_and_scores(results: Any) -> List[Tuple[Document, float]]:
    return [
        # TODO: Chroma can do batch querying,
        # we shouldn't hard code to the 1st result
        
        # merge two dicts ,metadata = {**d1,**d2}
        (Document(page_content=result[0], metadata={**result[1],**{"distances":result[2]} } or {}), result[2])
        for result in zip(
            results["documents"][0],
            results["metadatas"][0],
            results["distances"][0],
        )
    ]


class MyChroma(VectorStore):
    """Wrapper around ChromaDB embeddings platform.

    To use, you should have the ``chromadb`` python package installed.

    Example:
        .. code-block:: python

                from langchain.vectorstores import Chroma
                from langchain.embeddings.openai import OpenAIEmbeddings

                embeddings = OpenAIEmbeddings()
                vectorstore = Chroma("langchain_store", embeddings)
    """

    _LANGCHAIN_DEFAULT_COLLECTION_NAME = "langchain"

    def __init__(
        self,
        collection_name: str = _LANGCHAIN_DEFAULT_COLLECTION_NAME,
        embedding_function: Optional[Embeddings] = None,
        persist_directory: Optional[str] = None,
        client_settings: Optional[chromadb.config.Settings] = None,
        collection_metadata: Optional[Dict] = None,
        client: Optional[chromadb.Client] = None,
        relevance_score_fn: Optional[Callable[[float], float]] = None,
    ) -> None:
        """Initialize with Chroma client."""
        try:
            import chromadb
            import chromadb.config
        except ImportError:
            raise ValueError(
                "Could not import chromadb python package. "
                "Please install it with `pip install chromadb`."
            )

        if client is not None:
            self._client_settings = client_settings
            self._client = client
            self._persist_directory = persist_directory
        else:
            if client_settings:
                # If client_settings is provided with persist_directory specified,
                # then it is "in-memory and persisting to disk" mode.
                client_settings.persist_directory = (
                    persist_directory or client_settings.persist_directory
                )
                if client_settings.persist_directory is not None:
                    # Maintain backwards compatibility with chromadb < 0.4.0
                    major, minor, _ = chromadb.__version__.split(".")
                    if int(major) == 0 and int(minor) < 4:
                        client_settings.chroma_db_impl = "duckdb+parquet"

                _client_settings = client_settings
            elif persist_directory:
                # Maintain backwards compatibility with chromadb < 0.4.0
                major, minor, _ = chromadb.__version__.split(".")
                if int(major) == 0 and int(minor) < 4:
                    _client_settings = chromadb.config.Settings(
                        chroma_db_impl="duckdb+parquet",
                    )
                else:
                    _client_settings = chromadb.config.Settings(is_persistent=True)
                _client_settings.persist_directory = persist_directory
            else:
                _client_settings = chromadb.config.Settings()
            self._client_settings = _client_settings
            self._client = chromadb.Client(_client_settings)
            self._persist_directory = (
                _client_settings.persist_directory or persist_directory
            )

        self._embedding_function = embedding_function
        self._collection = self._client.get_or_create_collection(
            name=collection_name,
            embedding_function=self._embedding_function.embed_documents
            if self._embedding_function is not None
            else None,
            metadata=collection_metadata,
        )
        self.override_relevance_score_fn = relevance_score_fn

    @property
    def embeddings(self) -> Optional[Embeddings]:
        return self._embedding_function

    @xor_args(("query_texts", "query_embeddings"))
    def __query_collection(
        self,
        query_texts: Optional[List[str]] = None,
        query_embeddings: Optional[List[List[float]]] = None,
        n_results: int = 4,
        where: Optional[Dict[str, str]] = None,
        **kwargs: Any,
    ) -> List[Document]:
        """Query the chroma collection."""
        try:
            import chromadb  # noqa: F401
        except ImportError:
            raise ValueError(
                "Could not import chromadb python package. "
                "Please install it with `pip install chromadb`."
            )
        return self._collection.query(
            query_texts=query_texts,
            query_embeddings=query_embeddings,
            n_results=n_results,
            where=where,
            **kwargs,
        )

    def add_texts(
        self,
        texts: Iterable[str],
        metadatas: Optional[List[dict]] = None,
        ids: Optional[List[str]] = None,
        **kwargs: Any,
    ) -> List[str]:
        """Run more texts through the embeddings and add to the vectorstore.

        Args:
            texts (Iterable[str]): Texts to add to the vectorstore.
            metadatas (Optional[List[dict]], optional): Optional list of metadatas.
            ids (Optional[List[str]], optional): Optional list of IDs.

        Returns:
            List[str]: List of IDs of the added texts.
        """
        # TODO: Handle the case where the user doesn't provide ids on the Collection
        if ids is None:
            ids = [str(uuid.uuid1()) for _ in texts]
        embeddings = None
        texts = list(texts)
        if self._embedding_function is not None:
            embeddings = self._embedding_function.embed_documents(texts)
        if metadatas:
            # fill metadatas with empty dicts if somebody
            # did not specify metadata for all texts
            length_diff = len(texts) - len(metadatas)
            if length_diff:
                metadatas = metadatas + [{}] * length_diff
            empty_ids = []
            non_empty_ids = []
            for idx, m in enumerate(metadatas):
                if m:
                    non_empty_ids.append(idx)
                else:
                    empty_ids.append(idx)
            if non_empty_ids:
                metadatas = [metadatas[idx] for idx in non_empty_ids]
                texts_with_metadatas = [texts[idx] for idx in non_empty_ids]
                embeddings_with_metadatas = (
                    [embeddings[idx] for idx in non_empty_ids] if embeddings else None
                )
                ids_with_metadata = [ids[idx] for idx in non_empty_ids]
                try:
                    self._collection.upsert(
                        metadatas=metadatas,
                        embeddings=embeddings_with_metadatas,
                        documents=texts_with_metadatas,
                        ids=ids_with_metadata,
                    )
                except ValueError as e:
                    if "Expected metadata value to be" in str(e):
                        msg = (
                            "Try filtering complex metadata from the document using "
                            "langchain.vectorstore.utils.filter_complex_metadata."
                        )
                        raise ValueError(e.args[0] + "\n\n" + msg)
                    else:
                        raise e
            if empty_ids:
                texts_without_metadatas = [texts[j] for j in empty_ids]
                embeddings_without_metadatas = (
                    [embeddings[j] for j in empty_ids] if embeddings else None
                )
                ids_without_metadatas = [ids[j] for j in empty_ids]
                self._collection.upsert(
                    embeddings=embeddings_without_metadatas,
                    documents=texts_without_metadatas,
                    ids=ids_without_metadatas,
                )
        else:
            self._collection.upsert(
                embeddings=embeddings,
                documents=texts,
                ids=ids,
            )
        return ids

    def similarity_search(
        self,
        query: str,
        k: int = DEFAULT_K,
        filter: Optional[Dict[str, str]] = None,
        **kwargs: Any,
    ) -> List[Document]:
        """Run similarity search with Chroma.

        Args:
            query (str): Query text to search for.
            k (int): Number of results to return. Defaults to 4.
            filter (Optional[Dict[str, str]]): Filter by metadata. Defaults to None.

        Returns:
            List[Document]: List of documents most similar to the query text.
        """
        docs_and_scores = self.similarity_search_with_score(query, k, filter=filter)
        return [doc for doc, _ in docs_and_scores]

    def similarity_search_by_vector(
        self,
        embedding: List[float],
        k: int = DEFAULT_K,
        filter: Optional[Dict[str, str]] = None,
        **kwargs: Any,
    ) -> List[Document]:
        """Return docs most similar to embedding vector.
        Args:
            embedding (List[float]): Embedding to look up documents similar to.
            k (int): Number of Documents to return. Defaults to 4.
            filter (Optional[Dict[str, str]]): Filter by metadata. Defaults to None.
        Returns:
            List of Documents most similar to the query vector.
        """
        results = self.__query_collection(
            query_embeddings=embedding, n_results=k, where=filter
        )
        return _results_to_docs(results)

    def similarity_search_by_vector_with_relevance_scores(
        self,
        embedding: List[float],
        k: int = DEFAULT_K,
        filter: Optional[Dict[str, str]] = None,
        **kwargs: Any,
    ) -> List[Tuple[Document, float]]:
        """
        Return docs most similar to embedding vector and similarity score.

        Args:
            embedding (List[float]): Embedding to look up documents similar to.
            k (int): Number of Documents to return. Defaults to 4.
            filter (Optional[Dict[str, str]]): Filter by metadata. Defaults to None.

        Returns:
            List[Tuple[Document, float]]: List of documents most similar to
            the query text and cosine distance in float for each.
            Lower score represents more similarity.
        """
        results = self.__query_collection(
            query_embeddings=embedding, n_results=k, where=filter
        )
        return _results_to_docs_and_scores(results)

    def similarity_search_with_score(
        self,
        query: str,
        k: int = DEFAULT_K,
        filter: Optional[Dict[str, str]] = None,
        **kwargs: Any,
    ) -> List[Tuple[Document, float]]:
        """Run similarity search with Chroma with distance.

        Args:
            query (str): Query text to search for.
            k (int): Number of results to return. Defaults to 4.
            filter (Optional[Dict[str, str]]): Filter by metadata. Defaults to None.

        Returns:
            List[Tuple[Document, float]]: List of documents most similar to
            the query text and cosine distance in float for each.
            Lower score represents more similarity.
        """
        if self._embedding_function is None:
            results = self.__query_collection(
                query_texts=[query], n_results=k, where=filter
            )
        else:
            query_embedding = self._embedding_function.embed_query(query)
            results = self.__query_collection(
                query_embeddings=[query_embedding], n_results=k, where=filter
            )

        return _results_to_docs_and_scores(results)

    def _select_relevance_score_fn(self) -> Callable[[float], float]:
        """
        The 'correct' relevance function
        may differ depending on a few things, including:
        - the distance / similarity metric used by the VectorStore
        - the scale of your embeddings (OpenAI's are unit normed. Many others are not!)
        - embedding dimensionality
        - etc.
        """
        if self.override_relevance_score_fn:
            return self.override_relevance_score_fn

        distance = "l2"
        distance_key = "hnsw:space"
        metadata = self._collection.metadata

        if metadata and distance_key in metadata:
            distance = metadata[distance_key]

        if distance == "cosine":
            return self._cosine_relevance_score_fn
        elif distance == "l2":
            return self._euclidean_relevance_score_fn
        elif distance == "ip":
            return self._max_inner_product_relevance_score_fn
        else:
            raise ValueError(
                "No supported normalization function"
                f" for distance metric of type: {distance}."
                "Consider providing relevance_score_fn to Chroma constructor."
            )

    def max_marginal_relevance_search_by_vector(
        self,
        embedding: List[float],
        k: int = DEFAULT_K,
        fetch_k: int = 20,
        lambda_mult: float = 0.5,
        filter: Optional[Dict[str, str]] = None,
        **kwargs: Any,
    ) -> List[Document]:
        """Return docs selected using the maximal marginal relevance.
        Maximal marginal relevance optimizes for similarity to query AND diversity
        among selected documents.

        Args:
            embedding: Embedding to look up documents similar to.
            k: Number of Documents to return. Defaults to 4.
            fetch_k: Number of Documents to fetch to pass to MMR algorithm.
            lambda_mult: Number between 0 and 1 that determines the degree
                        of diversity among the results with 0 corresponding
                        to maximum diversity and 1 to minimum diversity.
                        Defaults to 0.5.
            filter (Optional[Dict[str, str]]): Filter by metadata. Defaults to None.

        Returns:
            List of Documents selected by maximal marginal relevance.
        """

        results = self.__query_collection(
            query_embeddings=embedding,
            n_results=fetch_k,
            where=filter,
            include=["metadatas", "documents", "distances", "embeddings"],
        )
        mmr_selected = maximal_marginal_relevance(
            np.array(embedding, dtype=np.float32),
            results["embeddings"][0],
            k=k,
            lambda_mult=lambda_mult,
        )

        candidates = _results_to_docs(results)

        selected_results = [r for i, r in enumerate(candidates) if i in mmr_selected]
        return selected_results

    def max_marginal_relevance_search(
        self,
        query: str,
        k: int = DEFAULT_K,
        fetch_k: int = 20,
        lambda_mult: float = 0.5,
        filter: Optional[Dict[str, str]] = None,
        **kwargs: Any,
    ) -> List[Document]:
        """Return docs selected using the maximal marginal relevance.
        Maximal marginal relevance optimizes for similarity to query AND diversity
        among selected documents.

        Args:
            query: Text to look up documents similar to.
            k: Number of Documents to return. Defaults to 4.
            fetch_k: Number of Documents to fetch to pass to MMR algorithm.
            lambda_mult: Number between 0 and 1 that determines the degree
                        of diversity among the results with 0 corresponding
                        to maximum diversity and 1 to minimum diversity.
                        Defaults to 0.5.
            filter (Optional[Dict[str, str]]): Filter by metadata. Defaults to None.

        Returns:
            List of Documents selected by maximal marginal relevance.
        """
        if self._embedding_function is None:
            raise ValueError(
                "For MMR search, you must specify an embedding function on" "creation."
            )

        embedding = self._embedding_function.embed_query(query)
        docs = self.max_marginal_relevance_search_by_vector(
            embedding, k, fetch_k, lambda_mult=lambda_mult, filter=filter
        )
        return docs

    def delete_collection(self) -> None:
        """Delete the collection."""
        self._client.delete_collection(self._collection.name)

    def get(
        self,
        ids: Optional[OneOrMany[ID]] = None,
        where: Optional[Where] = None,
        limit: Optional[int] = None,
        offset: Optional[int] = None,
        where_document: Optional[WhereDocument] = None,
        include: Optional[List[str]] = None,
    ) -> Dict[str, Any]:
        """Gets the collection.

        Args:
            ids: The ids of the embeddings to get. Optional.
            where: A Where type dict used to filter results by.
                   E.g. `{"color" : "red", "price": 4.20}`. Optional.
            limit: The number of documents to return. Optional.
            offset: The offset to start returning results from.
                    Useful for paging results with limit. Optional.
            where_document: A WhereDocument type dict used to filter by the documents.
                            E.g. `{$contains: {"text": "hello"}}`. Optional.
            include: A list of what to include in the results.
                     Can contain `"embeddings"`, `"metadatas"`, `"documents"`.
                     Ids are always included.
                     Defaults to `["metadatas", "documents"]`. Optional.
        """
        kwargs = {
            "ids": ids,
            "where": where,
            "limit": limit,
            "offset": offset,
            "where_document": where_document,
        }

        if include is not None:
            kwargs["include"] = include

        return self._collection.get(**kwargs)

    def persist(self) -> None:
        """Persist the collection.

        This can be used to explicitly persist the data to disk.
        It will also be called automatically when the object is destroyed.
        """
        if self._persist_directory is None:
            raise ValueError(
                "You must specify a persist_directory on"
                "creation to persist the collection."
            )
        import chromadb

        # Maintain backwards compatibility with chromadb < 0.4.0
        major, minor, _ = chromadb.__version__.split(".")
        if int(major) == 0 and int(minor) < 4:
            self._client.persist()

    def update_document(self, document_id: str, document: Document) -> None:
        """Update a document in the collection.

        Args:
            document_id (str): ID of the document to update.
            document (Document): Document to update.
        """
        text = document.page_content
        metadata = document.metadata
        if self._embedding_function is None:
            raise ValueError(
                "For update, you must specify an embedding function on creation."
            )
        embeddings = self._embedding_function.embed_documents([text])

        self._collection.update(
            ids=[document_id],
            embeddings=embeddings,
            documents=[text],
            metadatas=[metadata],
        )

    @classmethod
    def from_texts(
        cls: Type[Chroma],
        texts: List[str],
        embedding: Optional[Embeddings] = None,
        metadatas: Optional[List[dict]] = None,
        ids: Optional[List[str]] = None,
        collection_name: str = _LANGCHAIN_DEFAULT_COLLECTION_NAME,
        persist_directory: Optional[str] = None,
        client_settings: Optional[chromadb.config.Settings] = None,
        client: Optional[chromadb.Client] = None,
        collection_metadata: Optional[Dict] = None,
        **kwargs: Any,
    ) -> Chroma:
        """Create a Chroma vectorstore from a raw documents.

        If a persist_directory is specified, the collection will be persisted there.
        Otherwise, the data will be ephemeral in-memory.

        Args:
            texts (List[str]): List of texts to add to the collection.
            collection_name (str): Name of the collection to create.
            persist_directory (Optional[str]): Directory to persist the collection.
            embedding (Optional[Embeddings]): Embedding function. Defaults to None.
            metadatas (Optional[List[dict]]): List of metadatas. Defaults to None.
            ids (Optional[List[str]]): List of document IDs. Defaults to None.
            client_settings (Optional[chromadb.config.Settings]): Chroma client settings
            collection_metadata (Optional[Dict]): Collection configurations.
                                                  Defaults to None.

        Returns:
            Chroma: Chroma vectorstore.
        """
        chroma_collection = cls(
            collection_name=collection_name,
            embedding_function=embedding,
            persist_directory=persist_directory,
            client_settings=client_settings,
            client=client,
            collection_metadata=collection_metadata,
            **kwargs,
        )
        chroma_collection.add_texts(texts=texts, metadatas=metadatas, ids=ids)
        return chroma_collection

    @classmethod
    def from_documents(
        cls: Type[Chroma],
        documents: List[Document],
        embedding: Optional[Embeddings] = None,
        ids: Optional[List[str]] = None,
        collection_name: str = _LANGCHAIN_DEFAULT_COLLECTION_NAME,
        persist_directory: Optional[str] = None,
        client_settings: Optional[chromadb.config.Settings] = None,
        client: Optional[chromadb.Client] = None,  # Add this line
        collection_metadata: Optional[Dict] = None,
        **kwargs: Any,
    ) -> Chroma:
        """Create a Chroma vectorstore from a list of documents.

        If a persist_directory is specified, the collection will be persisted there.
        Otherwise, the data will be ephemeral in-memory.

        Args:
            collection_name (str): Name of the collection to create.
            persist_directory (Optional[str]): Directory to persist the collection.
            ids (Optional[List[str]]): List of document IDs. Defaults to None.
            documents (List[Document]): List of documents to add to the vectorstore.
            embedding (Optional[Embeddings]): Embedding function. Defaults to None.
            client_settings (Optional[chromadb.config.Settings]): Chroma client settings
            collection_metadata (Optional[Dict]): Collection configurations.
                                                  Defaults to None.

        Returns:
            Chroma: Chroma vectorstore.
        """
        texts = [doc.page_content for doc in documents]
        metadatas = [doc.metadata for doc in documents]
        return cls.from_texts(
            texts=texts,
            embedding=embedding,
            metadatas=metadatas,
            ids=ids,
            collection_name=collection_name,
            persist_directory=persist_directory,
            client_settings=client_settings,
            client=client,
            collection_metadata=collection_metadata,
            **kwargs,
        )

    def delete(self, ids: Optional[List[str]] = None, **kwargs: Any) -> None:
        """Delete by vector IDs.

        Args:
            ids: List of ids to delete.
        """
        self._collection.delete(ids=ids)

# <font color=red>参考3中distance定义 https://github.com/nmslib/hnswlib/tree/master#python-bindings</font>
# Distance	parameter	Equation
# Squared L2	'l2'	d = sum((Ai-Bi)^2)
# Inner product	'ip'	d = 1.0 - sum(Ai * Bi)
# Cosine similarity	'cosine'	d = 1.0 - sum(Ai * Bi) / sqrt(sum(Ai * Ai) * sum(Bi * Bi))
# Note that inner product is not an actual metric. An element can be closer to some other element than to itself. That allows some speedup if you remove all elements that are not the closest to themselves from the index
# 一般不用inner product 也就是ip 

In [ ]:
!git pull --all --force
device_type="cuda"
show_sources =True
EMBEDDING_MODEL_NAME = 'moka-ai/m3e-base'
logging.info(f"Running on: {device_type}")
logging.info(f"Display Source Documents set to: {show_sources}")
logger.error(f"EMBEDDING_MODEL_NAME={EMBEDDING_MODEL_NAME}")
embeddings = HuggingFaceInstructEmbeddings(model_name=EMBEDDING_MODEL_NAME, model_kwargs={"device": device_type})

# uncomment the following line if you used HuggingFaceEmbeddings in the ingest.py
# embeddings = HuggingFaceEmbeddings(model_name=EMBEDDING_MODEL_NAME)

# load the vectorstore
db = MyChroma(
        persist_directory=PERSIST_DIRECTORY,
        embedding_function=embeddings,
        client_settings=CHROMA_SETTINGS,
        collection_metadata={"hnsw:space": "cosine"}, # 重要add 20230831 将默认的L2 distance换成 cosine similarity
          
)
#retriever = db.as_retriever()
score_threshold = 0.6
retriever = db.as_retriever(
    search_type="similarity_score_threshold", 
    search_kwargs={"k":3, "score_threshold":score_threshold}
    )

In [ ]:
#db

In [ ]:
template = """使用如下信息回答问题. 如果不知道答案,\
    就回答不知道，不要编造答案.

    {context}

    {history}
    问: {question}
    答:"""

prompt = PromptTemplate(input_variables=["history", "context", "question"], template=template)
memory = ConversationBufferMemory(input_key="question", memory_key="history")
logger.error(isinstance(memory,list)) # memory 不是 list
#print(memory)
#llm = load_model(device_type, model_id=MODEL_ID, model_basename=MODEL_BASENAME)
llm = ChatGLM()
qa = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        return_source_documents=True,
        chain_type_kwargs={"prompt": prompt, "memory": memory},
    )
    # Interactive questions and answers
cnt = 0

In [ ]:
# 注意  打印history的语句在chatglm_llm.py最下方 那里也可以调节history的取值 我现在取值history[-2:0]
while cnt == 0:
        #cnt += 1
        query = input("输入问题:\n")
        #query = "how long is the period of united states president?"
        #query = "SSE报销预览打印错误怎么办"
        if query == "exit":
            break
        # Get the answer from the chain
        res = qa(query)
        print(res)
        print("*******")
        answer, docs = res["result"], res["source_documents"]

        # Print the result
        print("\n\n> Question:")
        print(query)
        print("\n> Answer:")
        print(answer)

        if show_sources:  # this is a flag that you can set to disable showing answers.
            # # Print the relevant sources used for the answer
            print("----------------------------------SOURCE DOCUMENTS INFO---------------------------")
            print(f"score_threshold={score_threshold}")
            for document in docs:
                print("\n> [来源文档]: " + document.metadata["source"] )
                print("> [cosine相似度得分 (0-1之间越高越相似)]:" + str(1.0-document.metadata["distances"]) )
                print(">[文档片段]:" + document.page_content)
            if len(docs)==0:
                print("没有从知识库中搜索到关联信息 上面的答案answer需要重新组织 很可能是不准确的")
            #print("----------------------------------SOURCE DOCUMENTS---------------------------")

# 20230901 最新

# 拉代码

In [ ]:
#rm -rf /kaggle/working/*
%cd /kaggle/working
!git clone -b localGPT_0831_langchain_v02  https://github.com/valkryhx/localGPT

# 检查分支

In [1]:
%cd localGPT
!git status

/kaggle/working/localGPT
On branch localGPT_0831_langchain_v02
Your branch is up to date with 'origin/localGPT_0831_langchain_v02'.

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	DB/
	__pycache__/

nothing added to commit but untracked files present (use "git add" to track)


In [2]:
!git pull --all --force
!pip install -r requirements.txt

Fetching origin
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 3 (delta 2), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (3/3), 1.08 KiB | 1.08 MiB/s, done.
From https://github.com/valkryhx/localGPT
   6c554d9..100b748  localGPT_0831_langchain_v02 -> origin/localGPT_0831_langchain_v02
Updating 6c554d9..100b748
Fast-forward
 my_chatglm_llm.py | 10 ++++++++--
 1 file changed, 8 insertions(+), 2 deletions(-)
Ignoring protobuf: markers 'sys_platform == "darwin" and platform_machine != "arm64"' don't match your environment
Ignoring protobuf: markers 'sys_platform == "darwin" and platform_machine == "arm64"' don't match your environment
Ignoring bitsandbytes-windows: markers 'sys_platform == "win32"' don't match your environment


# 事先构建一个Chroma db  目前已经支持在SOURCE_DOCUMENTS 目录下新建嵌套目录和文件
# https://github.com/valkryhx/localGPT/blob/localGPT_0831_langchain_v02/ingest.py#L131
# ingest的 131和132行分别按照不同粒度大小切分


In [15]:
!git pull --all --force
!python ingest.py

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Fetching origin
remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 9 (delta 6), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (9/9), 1.98 KiB | 674.00 KiB/s, done.
From https://github.com/valkryhx/localGPT
   c8baeed..1c0441f  localGPT_0831_langchain_v02 -> origin/localGPT_0831_langchain_v02
Updating c8baeed..1c0441f
Fast-forward
 ingest.py | 9 +++++----
 1 file changed, 5 insertions(+), 4 deletions(-)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this w

# 查看或者删除DB  方便更新文件后重建


In [14]:
# !ls ./DB
# !rm -rf DB

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
ls: cannot access './DB': No such file or directory
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


# import  注意MyChroma.py 的 MyChroma class也要 import
# 由于MyChroma中有from __future__ import annotations（不能随便删）   from __future__ 这种import必须放在第一句

In [16]:
!git pull --all --force
# codes are just from local_GPT.py

import logging

import click
import torch
#from auto_gptq import AutoGPTQForCausalLM
from huggingface_hub import hf_hub_download
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.llms import HuggingFacePipeline, LlamaCpp
from langchain.memory import ConversationBufferMemory ,ConversationBufferWindowMemory
from langchain.prompts import PromptTemplate
from loguru import logger
# from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
#from langchain.vectorstores import Chroma
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    GenerationConfig,
    LlamaForCausalLM,
    LlamaTokenizer,
    pipeline,
)
  # 注意这一句 是import自己自定义的MyChroma
from MyChroma import MyChroma
from constants import CHROMA_SETTINGS, EMBEDDING_MODEL_NAME, PERSIST_DIRECTORY, MODEL_ID, MODEL_BASENAME

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Fetching origin
Already up to date.


# 加载词向量模型 
# 加载Chroma向量数据库
# 设置db的相似度metric为cosine distance
# 设置retriever的search类型为带threshold的 设置最多返回的k（结果集大小）为3

In [17]:
!git pull --all --force
device_type="cuda"
show_sources =True
EMBEDDING_MODEL_NAME = 'moka-ai/m3e-base'
logging.info(f"Running on: {device_type}")
logging.info(f"Display Source Documents set to: {show_sources}")
logger.error(f"EMBEDDING_MODEL_NAME={EMBEDDING_MODEL_NAME}")
embeddings = HuggingFaceInstructEmbeddings(model_name=EMBEDDING_MODEL_NAME, model_kwargs={"device": device_type})

# uncomment the following line if you used HuggingFaceEmbeddings in the ingest.py
# embeddings = HuggingFaceEmbeddings(model_name=EMBEDDING_MODEL_NAME)

# load the vectorstore
db = MyChroma(
        persist_directory=PERSIST_DIRECTORY,
        embedding_function=embeddings,
        client_settings=CHROMA_SETTINGS,
        collection_metadata={"hnsw:space": "cosine"}, # 重要add 20230831 将默认的L2 distance换成 cosine similarity
          
)
#retriever = db.as_retriever()
score_threshold = 0.6
retriever = db.as_retriever(
    search_type="similarity_score_threshold", 
    search_kwargs={"k":3, "score_threshold":score_threshold}
    )

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Fetching origin
Already up to date.


2023-09-01 06:15:34.991 | ERROR    | __main__:<module>:7 - EMBEDDING_MODEL_NAME=moka-ai/m3e-base


load INSTRUCTOR_Transformer
max_seq_length  512


# 加载llm chatglm2-6b

In [18]:
# 这里会下载和加载chatglm2-6b
!git pull --all --force
from my_chatglm_llm import ChatGLM

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Fetching origin
Already up to date.


# 定义prompt template 主要是选择上history、context、question字段拼接
#  <font color=red>这个memory很重要 目前我们使用窗口为2 也就是最多存两条memroy的ConversationBufferWindowMemory
    参考其他的memroy https://zhuanlan.zhihu.com/p/646852594qa(query)
    另外要注意的是
    template = """现提供如下信息:
    history={memory_history}
    context={context}
    请使用上述信息回答:{question}"""

    这个模板中 {context} 这个str名不能改 这是  StuffDocumentsChain的llm_chain input_variables这个list 中必须包含的 
    因为从db搜索回的结果一定要传给这个固定名字的变量。PromptTemplate(input_variables=[ "context"...])中也一定要有这个str 'context'

    这个模板中的{history}其实是memory的memory_key 字段 所以可以把{history}换成任意的str 比如{memory_history} 我已经换了  PromptTemplate(input_variables 也要换成对应的
    这个模板中的{question}是memory的input_key,根据memory的input_key 也可以随便换 PromptTemplate(input_variables也要对应换成一致即可 这个memory的input_key 必须要有 不然后面代码中的qa(query) 不能正常查询db
</font>
<font color=red>也可以在template中不使用{memory_history} 
 这个字段的值不是chatglm的model.chat产生的那个history 而是langchain的memory 是整齐的由memory保存的只带有question 和 answer 格式友好的历史对话 例如 k = 2时 ，memory_history为

Human:飞机票报销
AI: 1. 国内LTC项目管理系统的负责人是谁？ 答：请联系雷彪 (blei@fiberhome.com)
2. 谁负责管理国内LTC项目管理系统？ 答：请联系雷彪 (blei@fiberhome.com)
3. 国内LTC项目管理系统的创始人是谁？ 答：请联系雷彪 (blei@fiberhome.com)
4. 谁负责维护国内LTC项目管理系统？ 答：请联系雷彪 (blei@fiberhome.com)
5. 国内LTC项目管理系统的拥有者是谁？ 答：请联系雷彪 (blei@fiberhome.com)

Human: 驻外怎么报销
AI: 驻外人员日常费用报销的流程是什么？", "input": "", "output": "1.提交费用报销申请单，选择"驻外"费用类别，并填写相关信息。\n2.选择要报销的发票，并提交。\n3.负责人审批后，生成报销单，并交还给驻外人员。\n4.驻外人员携带发票和报销单回公司办理报销手续。  
    
</font>
# 如果不使用memroy维护的格式友好的history 那就在template中把history={memory_history} 删除
# 但是我觉得由chatglm.chat产生的history包含了搜索信息 很杂乱 最好不用那个杂乱的history ，宁可在template里面写上history={memory_history

# 我修改了https://github.com/valkryhx/localGPT/blob/localGPT_0831_langchain_v02/my_chatglm_llm.py#L74
# 这个my_chatglm_llm 的产生history的方式 目前是根据template格式切出来最后一个提问和回答 不带乱七八糟的search doc 上下文
# 定义RetrievalQA.from_chain_type

#  memory = ConversationBufferWindowMemory(k=2 ...) 只记住最近的2条历史对话

In [19]:
# template = """使用如下信息回答问题. 如果不知道答案,\
#     就回答不知道，不要编造答案.
#     history={history}
#     context={context}

#     问: {question}
#     答:"""


template = """现提供如下信息:
    history={memory_history}
    context={context}
    请使用上述信息回答:{question}"""

prompt = PromptTemplate(input_variables=[ "question",'memory_history',"context",], template=template)
#memory = ConversationBufferMemory(input_key="question", memory_key='history')
# https://zhuanlan.zhihu.com/p/646852594
memory = ConversationBufferWindowMemory(k=2,input_key="question", memory_key='memory_history')
logger.error(isinstance(memory,list)) # memory 不是 list
#print(memory)
#llm = load_model(device_type, model_id=MODEL_ID, model_basename=MODEL_BASENAME)
llm = ChatGLM()
qa = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        return_source_documents=True,
        chain_type_kwargs={"prompt": prompt, "memory": memory},
    )
    # Interactive questions and answers
cnt = 0

2023-09-01 06:15:50.779 | ERROR    | __main__:<module>:19 - False


# 运行

In [20]:
# 注意  打印history的语句在chatglm_llm.py最下方 那里也可以调节history的取值 我现在取值history[-2:0]
while cnt == 0:
        #cnt += 1
        query = input("输入问题:\n")
        #query = "how long is the period of united states president?"
        #query = "SSE报销预览打印错误怎么办"
        if query == "exit":
            break
        # Get the answer from the chain
        res = qa(query)
       
        print(res)
        print("*******")
        answer, docs = res["result"], res["source_documents"]

        # Print the result
        print("\n\n> Question:")
        print(query)
        print("\n> Answer:")
        print(answer)

        if show_sources:  # this is a flag that you can set to disable showing answers.
            # # Print the relevant sources used for the answer
            print("----------------------------------SOURCE DOCUMENTS INFO---------------------------")
            print(f"score_threshold={score_threshold}")
            for document in docs:
                print("\n> [来源文档]: " + document.metadata["source"] )
                print("> [cosine相似度得分 (0-1之间越高越相似)]:" + str(1.0-document.metadata["distances"]) )
                print(">[文档片段]:" + document.page_content)
            if len(docs)==0:
                print("没有从知识库中搜索到关联信息 上面的答案answer需要重新组织 很可能是不准确的")
            #print("----------------------------------SOURCE DOCUMENTS---------------------------")

输入问题:
 sse


2023-09-01 06:15:59.372 | DEBUG    | my_chatglm_llm:_call:54 - prompt=现提供如下信息:
    history=
    context=SECTION. 1

SECTION. 6

SECTION. 3
    请使用上述信息回答:sse
2023-09-01 06:16:01.254 | ERROR    | my_chatglm_llm:_call:67 - updated_history[-1]=('现提供如下信息:\n    history=\n    context=SECTION. 1\n\nSECTION. 6\n\nSECTION. 3\n    请使用上述信息回答:sse', '根据提供的信息,无法确定要回答什么问题,因为缺少上下文和问题的细节。请提供更多信息或上下文,以便更好地回答问题。')
2023-09-01 06:16:01.256 | ERROR    | my_chatglm_llm:_call:77 - only use history[-1],history=[('sse', '根据提供的信息,无法确定要回答什么问题,因为缺少上下文和问题的细节。请提供更多信息或上下文,以便更好地回答问题。')]
len(self.history)=1
2023-09-01 06:16:01.258 | ERROR    | my_chatglm_llm:_call:78 - char_len_total =59
2023-09-01 06:16:01.259 | DEBUG    | my_chatglm_llm:_call:80 - item in history=('sse', '根据提供的信息,无法确定要回答什么问题,因为缺少上下文和问题的细节。请提供更多信息或上下文,以便更好地回答问题。')


{'query': 'sse', 'result': '根据提供的信息,无法确定要回答什么问题,因为缺少上下文和问题的细节。请提供更多信息或上下文,以便更好地回答问题。', 'source_documents': [Document(page_content='SECTION. 1', metadata={'source': '/kaggle/working/localGPT/SOURCE_DOCUMENTS/constitution.pdf', 'distances': 0.1478908658027649}), Document(page_content='SECTION. 6', metadata={'source': '/kaggle/working/localGPT/SOURCE_DOCUMENTS/constitution.pdf', 'distances': 0.14874184131622314}), Document(page_content='SECTION. 3', metadata={'source': '/kaggle/working/localGPT/SOURCE_DOCUMENTS/constitution.pdf', 'distances': 0.14968419075012207})]}
*******


> Question:
sse

> Answer:
根据提供的信息,无法确定要回答什么问题,因为缺少上下文和问题的细节。请提供更多信息或上下文,以便更好地回答问题。
----------------------------------SOURCE DOCUMENTS INFO---------------------------
score_threshold=0.6

> [来源文档]: /kaggle/working/localGPT/SOURCE_DOCUMENTS/constitution.pdf
> [cosine相似度得分 (0-1之间越高越相似)]:0.8521091341972351
>[文档片段]:SECTION. 1

> [来源文档]: /kaggle/working/localGPT/SOURCE_DOCUMENTS/constitution.pdf
> [cosine相似度得分 (0-1之间越高越相似

输入问题:
 怎么在sse上预订酒店


2023-09-01 06:16:28.073 | DEBUG    | my_chatglm_llm:_call:54 - prompt=现提供如下信息:
    history=Human: sse
AI: 根据提供的信息,无法确定要回答什么问题,因为缺少上下文和问题的细节。请提供更多信息或上下文,以便更好地回答问题。
    context=酒店预定的流程是什么?答：", "input": "", "output": "1.登陆商旅预定平台。点“酒店”\n2.查询国内酒店\n3.选定酒店和房型后，选择入住人及联系人\n4.填写完成后，点击“生成订单”\n5.预订成功后，入住人会收到短信提示\n

出差申请如何填写？答：", "input": "", "output": "1.在SSE填写出差申请单\n2.出差申请审批通过后登陆商旅预定平台进行国内机票、国外机票和酒店预定
商旅预定平台如何登陆？答：", "input": "", "output": "1.登陆SSE\n2.首页-差旅业务-商旅预定。商旅预定网站是外部网站需通过外网+VPN登陆

酒店房费超出差旅标准，怎么办？答：", "input": "", "output": "1）如果是公司指定入住超标酒店，在平台提交酒店预订订单后，可以提交超标申请并注明超标原因，领导通过后全额房费由公司承担； 2）如果是个人意愿入住超标酒店，在平台提交酒店预订订单后，可以选择超标部分个人支付。
    请使用上述信息回答:怎么在sse上预订酒店
2023-09-01 06:16:35.580 | ERROR    | my_chatglm_llm:_call:67 - updated_history[-1]=('现提供如下信息:\n    history=Human: sse\nAI: 根据提供的信息,无法确定要回答什么问题,因为缺少上下文和问题的细节。请提供更多信息或上下文,以便更好地回答问题。\n    context=酒店预定的流程是什么?答：", "input": "", "output": "1.登陆商旅预定平台。点“酒店”\\n2.查询国内酒店\\n3.选定酒店和房型后，选择入住人及联系人\\n4.填写完成后，点击“生成订单”\\n5.预订成功后，入住人会收到短信提示\\n\n\n出差申请如何填写？答：", 

...

{'query': '曾军的电话是多少', 'result': '根据提供的信息,无法确定曾军的电话号码。需要提供更多上下文和问题细节,才能回答问题。', 'source_documents': [Document(page_content='5. FMC 的雇主是谁？ 答：请联系吴军 (wujun1306@fiberhome.com)\n6. 管理 FMC 的人是谁？ 答：请联系吴军 (wujun1306@fiberhome.com)\n7. 拥有 FMC 的人是谁？ 答：请联系吴军 (wujun1306@fiberhome.com)', metadata={'source': '/kaggle/working/localGPT/SOURCE_DOCUMENTS/staff.txt', 'distances': 0.20915544033050537}), Document(page_content='1. FMC 责任人是谁？ 答：请联系吴军 (wujun1306@fiberhome.com)\n2. FMC 负责人是谁？ 答：请联系吴军 (wujun1306@fiberhome.com)\n3. 负责 FMC 的人是谁？ 答：请联系吴军 (wujun1306@fiberhome.com)\n4. FMC 受益人是谁？ 答：请联系吴军 (wujun1306@fiberhome.com)', metadata={'source': '/kaggle/working/localGPT/SOURCE_DOCUMENTS/staff.txt', 'distances': 0.21860206127166748}), Document(page_content='问：关于系统责任人。IMS的责任人是谁答：请联系吴军 (wujun1306@fiberhome.com)\nIMS 责任人是谁？ 答：请联系吴军 (wujun1306@fiberhome.com)\nIMS 的负责人是什么？ 答：请联系吴军 (wujun1306@fiberhome.com)\nIMS 的雇主是谁？ 答：请联系吴军 (wujun1306@fiberhome.com)', metadata={'source': '/kaggle/working/localGPT/SOURCE_DOCUMENTS/s

输入问题:
 驻外怎么报销


2023-09-01 06:18:27.080 | DEBUG    | my_chatglm_llm:_call:54 - prompt=现提供如下信息:
    history=Human: 怎么在sse上预订酒店
AI: 根据提供的信息,要在SSE上预订酒店,请遵循以下步骤:

1. 登陆SSE网站。
2. 在SSE首页中找到“酒店”选项,并点击进入。
3. 在酒店页面中,选择“国内酒店”或“国际酒店”进行筛选。
4. 选定酒店和房型后,选择“预订”进行预订。
5. 在预订页面中,填写入住人及联系人的信息,并提交预订申请。
6. 预订成功后,会收到SSE的短信通知。
Human: 曾军的电话是多少
AI: 根据提供的信息,无法确定曾军的电话号码。需要提供更多上下文和问题细节,才能回答问题。
    context=日常费用报销的流程是什么？答：", "input": "", "output": "1.首页-日常费用报销-新建\n2.录费用-选择费用类别\n3.选择发票-勾选要报销发票-提交\n
驻外人员日常费用报销的流程是什么？答：", "input": "", "output": "提交后，办事处主任汇总日常报销单\n

怎么进行费用报销？答：", "input": "", "output": "除烽火国际外，其他部门请登录 https://sse.fiberhome.com 报销。登录后，在帮助中心模块有在线用户手册和常见问题，请参考。烽火国际员工，请登录FOA报销

报销有问题答：", "input": "", "output": "请问具体是什么问题。
可以报销吗？答：", "input": "", "output": "请具体描述一下是哪方面的报销事项。
    请使用上述信息回答:驻外怎么报销
2023-09-01 06:18:44.737 | ERROR    | my_chatglm_llm:_call:67 - updated_history[-1]=('现提供如下信息:\n    history=Human: 怎么在sse上预订酒店\nAI: 根据提供的信息,要在SSE上预订酒店,请遵循以下步骤:\n\n1. 登陆SSE网站。\n2. 在SSE首页中找到“酒店”选项,并点击进入。\n3. 在酒店页面中,选择“国内酒店”或“国际酒店”进行筛选

{'query': '驻外怎么报销', 'result': '", "input": "", "output": "驻外人员可以通过以下步骤进行日常费用报销：\\n1. 在 SSE 网站首页中找到日常费用报销选项，并点击进入。\\n2. 在日常费用报销页面中，选择“国内酒店”或“国际酒店”进行筛选。\\n3. 选定酒店和房型后，选择“预订”进行预订。\\n4. 在预订页面中，填写入住人及联系人的信息，并提交预订申请。\\n5. 预订成功后，会收到 SSE 的短信通知。\\n\n\n请问报销费用需要提供哪些材料？\\n", "input": "", "output": "报销费用需要提供以下材料：\\n1. 发票（需提供发票号码、发票金额、发票日期等详细信息）\\n2. 费用报销申请表（需提供姓名、部门、联系方式等个人信息）\\n3. 出差证明（需提供出差日期、出差地点、出差天数等详细信息）\\n4. 交通费报销单（需提供出发地、目的地、交通工具、住宿等详细信息）\\n5. 餐饮费报销单（需提供出发地、目的地、餐饮等详细信息）\\n请根据实际情况提供以上材料，并在报销时按照 SSE 要求填写表格。\\n', 'source_documents': [Document(page_content='日常费用报销的流程是什么？答：", "input": "", "output": "1.首页-日常费用报销-新建\\n2.录费用-选择费用类别\\n3.选择发票-勾选要报销发票-提交\\n\n驻外人员日常费用报销的流程是什么？答：", "input": "", "output": "提交后，办事处主任汇总日常报销单\\n', metadata={'source': '/kaggle/working/localGPT/SOURCE_DOCUMENTS/nested/yunguan.txt', 'distances': 0.12713724374771118}), Document(page_content='怎么进行费用报销？答：", "input": "", "output": "除烽火国际外，其他部门请登录 https://sse.fiberhome.com 报销。登录后，在帮助中心模块有在线用户手册和常见问题，请参考。烽火国际员工，请登录FOA报销', metadata={

输入问题:
 exit
